In [43]:
import nltk
import pandas as pd

nltk.download('punkt')

""" There are many features to determine if a text is from an author, for example:

- Word Frecuency
- Vocabulary Richness
- Verb C
- N-Grams
- etc.

In this case, I will use Word Frecuency
"""

# 1st, I will check the available data. This data contains the texts with their respective Author. 
texts_with_author = pd.read_csv("data/train.csv")
texts_with_author.head()


[nltk_data] Downloading package punkt to
[nltk_data]     /home/arturoaviles/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [46]:
# The ids are not important, we need to group and join all the texts by author
# The stop words inside this texts will help beacuse we are not trying to understand the intetion of the author.
# Every person uses this stop words with more or less frecuency than others.
texts_groupby_author = texts_with_author.groupby("author")["text"].apply(' '.join).reset_index()

texts_groupby_author

,author,text
0,EAP,"This process, however, afforded me no means of..."
1,HPL,It never once occurred to me that the fumbling...
2,MWS,How lovely is spring As we looked from Windsor...


In [48]:
# We transform every text to lowercase to eliminate the tokenization of same words with capitalized letters.
texts_groupby_author["text"] = texts_groupby_author.text.str.lower()

texts_groupby_author

,author,text
0,EAP,"this process, however, afforded me no means of..."
1,HPL,it never once occurred to me that the fumbling...
2,MWS,how lovely is spring as we looked from windsor...


In [65]:
# We can use a dict to save the word frequencies for each author
word_frequencies_by_author = {}

for _, row in texts_groupby_author.iterrows():
    author = row["author"]
    text = row["text"]
    tokens = nltk.tokenize.word_tokenize(text)
    frequency = nltk.FreqDist(tokens)
    word_frequencies_by_author[author] = frequency

In [72]:
sentence = "Still, as I urged our leaving Ireland with such inquietude and impatience, my father thought it best to yield."
sentence = sentence.lower()
sentence_tokens = nltk.tokenize.word_tokenize(sentence)

for author in word_frequencies_by_author.keys():
    total = 0
    for word in sentence_tokens:
        total += word_frequencies_by_author[author].freq(word) 
    print(total, author)

0.2824342312149186 EAP
0.23157307827166596 HPL
0.27968435547081877 MWS


In [147]:
# Create a new dataframe to save the data
dataframe_with_frequencies = pd.DataFrame(columns=('author', 'frequency', 'sentence'))
dataframe_with_frequencies.head()

,author,frequency,sentence


In [148]:

# Open the test file and iterate on it
test = pd.read_csv("data/test.csv")
for iter_num, row in test.iterrows():
    sentence = row["text"] # Get sentence
    sentence = sentence.lower() # Str to lower
    sentence_tokens = nltk.tokenize.word_tokenize(sentence) # Tokenize test words
    
    # Get the author and probability of authorship attribution
    best_frequency_author = ""
    best_frequency_by_author = 0
    for author in word_frequencies_by_author.keys():
        total = 0
        for word in sentence_tokens:
            total += word_frequencies_by_author[author].freq(word) 
        #print(total, author)
        if(total > best_frequency_by_author):
            best_frequency_author = author
            best_frequency_by_author = total
        
    # Add a new row to the dataframe
    dataframe_with_frequencies.loc[iter_num+1] = [best_frequency_author, best_frequency_by_author, sentence]

In [149]:
dataframe_with_frequencies.head()

# Save the dataframe to a CSV file
dataframe_with_frequencies.to_csv("results.csv")